# VERIFICAÇÃO DOS ENDEREÇOS DE ACORDO COM OS CORREIOS

In [1]:
import os
from json import loads, dumps
from time import sleep

import zeep

CORREIOS = zeep.Client('https://apps.correios.com.br/SigepMasterJPA/AtendeClienteService/AtendeCliente?wsdl')

CORREIOS_DIR = os.path.join(os.getcwd(), '..', 'LOTES', 'PRIMEIRO', 'correios')

def get_postaladdress(postal_code: str) -> dict:
    """
    
    """
    try:
        file_name = f'{os.path.join(CORREIOS_DIR, postal_code)}.json'
        
        if os.path.isfile(file_name):
            with open(file_name, 'rt') as file_handler:
                data = loads(file_handler.read())
        
        else:
            print('>> Consultando o CEP: {postal_code}')
            original = CORREIOS.service.consultaCEP(postal_code)

            data = {
                'bairro': original['bairro'],
                'cep': original['cep'],
                'cidade': original['cidade'],
                'complemento2': original['complemento2'],
                'end': original['end'],
                'uf': original['uf'],
                'unidadesPostagem': original['unidadesPostagem'],
            }
            
            with open(file_name, 'wt') as file_handler:
                file_handler.write(dumps(data, indent=4))

        return data
    except Exception as err:
        print(f'!!! Deu erro no CEP: [{postal_code}] - {err} !!!')

## Processamento no site dos Correios

In [2]:
import os
import re
from json import loads, dumps
from pprint import pprint

REGEX = re.compile('([^ ]*)[ ](.*)([ ]\d*)?$')

WORK_DIR = os.path.join(os.getcwd(), 'FINAL_RESULT')

for file_name in [os.path.join(WORK_DIR, each) for each in os.listdir(WORK_DIR)]:
    while True:
        try:
            with open(file_name, 'rt') as file_handler:
                data = loads(file_handler.read())

            formatted_cep = data['pre_preparation_address']['postal_code']
            formatted_cep = ('0' * (8 - len(formatted_cep))) + formatted_cep

            if 'correios' not in data or formatted_cep != data['correios']['cep']:
                data['correios'] = get_postaladdress(formatted_cep)

                parts = REGEX.search(data['correios']['end'])

                data['address'] = {
                    'postal_code': data['correios']['cep'],
                    'type': parts[1],
                    'full_name': parts[2],
                    'number': data['pre_preparation_address']['number'],
                    'complement': data['pre_preparation_address']['complement'], 
                    'reference': data['pre_preparation_address']['reference'], 
                    'district': data['correios']['bairro'],
                    'city': data['correios']['cidade'],
                    'state': data['correios']['uf'],
                    'country': 'BRASIL'
                }

                formatted_address = (
                    f'{data["address"]["type"]} {data["address"]["full_name"]}, ' 
                    f'{data["address"]["number"] or "???"}'
                    f'{", " + data["address"]["complement"] + " - " if data["address"]["complement"] else " - "}'
                    f'{(data["address"]["district"] + ", ") if data["address"]["district"] else "" }'
                    f'{data["address"]["city"]}'
                )   

                data['address']['formatted_address'] = formatted_address

                with open(file_name, 'wt') as file_handler:
                    file_handler.write(dumps(data, indent=4))
                      
                break
        except Exception as err:
            pass
              

print('*** CEPs COLETADOS COM SUCESSO ***')


*** CEPs COLETADOS COM SUCESSO ***
